In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="TeieUHWBZSJGpBxZWh10")
project = rf.workspace("mishmeria").project("failure-pallet-detection-94gji")
version = project.version(2)
dataset = version.download("yolov8")

In [2]:
from ultralytics import YOLO
model = YOLO("yolov8n.pt")

In [ ]:
import torch
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
print("GPU in use:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU only")


In [ ]:
model.train(
    data="failure-pallet-detection-2/data.yaml",
    epochs=100,          # enough to converge on 3.8k images
    imgsz=640,           # matches Roboflow preprocessing
    batch=16,            # adjust if OOM, try 8 if needed
    patience=20,         # early stopping if no improvement
    optimizer="AdamW",   # better on small/medium datasets
    device=0,            # force CUDA GPU
    workers=2,           # safe for Jupyter; increase if CPU is free
    project="runs/detect", name="pallet_defect_v1"
)

In [ ]:
import cv2
from ultralytics import YOLO
model = YOLO("D:/ClassifyModel/runs/detect/pallet_defect_v1/weights/best.pt")
cap = cv2.VideoCapture("C:/Program Files (x86)/HIK IN/HIK IN/storage/2025-08-26 14.34.41.mp4")

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    results = model(frame, conf=0.25)
    annotated_frame = results[0].plot()

    cv2.imshow("YOLO Detection", annotated_frame)
    if cv2.waitKey(1) & 0xFF == ord("q"):  # press q to quit
        break

cap.release()
cv2.destroyAllWindows()